In [1]:
!pip install dask-ml --quiet
!pip install -U ipykernel --quiet

In [2]:
!pip install dask[complete] --quiet

In [3]:
!pip install dask distributed --upgrade --quiet

In [4]:
!pip install dask-ml[complete] --quiet

In [5]:
!pip install dask-ml decomposition

  ERROR: Could not find a version that satisfies the requirement decomposition (from versions: none)
ERROR: No matching distribution found for decomposition


In [6]:
import dask_ml

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib
from dask_ml.model_selection import train_test_split
import dask.dataframe as dd
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

C:\Users\Brian\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\array\random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [8]:
from dask.distributed import Client, progress
from dask_ml.model_selection import train_test_split
import dask.dataframe as dd


In [9]:
client = Client(n_workers=32, threads_per_worker=4, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:65105 Dashboard: http://127.0.0.1:65104/status,Cluster Workers: 32 Cores: 128 Memory: 64.00 GB


In [10]:
PATH = 'https://raw.githubusercontent.com/brianfarrelly/data/master/parkinsons_updrs.csv'
parkinson = dd.read_csv(PATH)


In [11]:
park = parkinson.copy()
park.persist().compute()
park.compute()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,42,61,0,142.7900,22.485,33.485,0.00406,0.000031,0.00167,0.00168,...,0.160,0.00973,0.01133,0.01549,0.02920,0.025137,22.369,0.64215,0.55314,0.21367
5871,42,61,0,149.8400,21.988,32.988,0.00297,0.000025,0.00119,0.00147,...,0.215,0.01052,0.01277,0.01904,0.03157,0.011927,22.886,0.52598,0.56518,0.12621
5872,42,61,0,156.8200,21.495,32.495,0.00349,0.000025,0.00152,0.00187,...,0.244,0.01371,0.01456,0.01877,0.04112,0.017701,25.065,0.47792,0.57888,0.14157
5873,42,61,0,163.7300,21.007,32.007,0.00281,0.000020,0.00128,0.00151,...,0.131,0.00693,0.00870,0.01307,0.02078,0.007984,24.422,0.56865,0.56327,0.14204


In [ ]:
park.describe()

In [12]:
# Y is the target variable

Y = park["total_UPDRS"]


# X is the feature set

X = park[['age', 'test_time', 'Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP', 'Jitter:PPQ5', 
          'Jitter:DDP', 'Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 
          'NHR', 'HNR', 'RPDE', 'DFA', 'PPE', 'Shimmer:APQ11', 'Shimmer:DDA']]


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)



X_train = X_train.persist()
X_test =  X_test.persist()
y_train = y_train.persist()
y_test = y_test.persist()


In [13]:
X_train.compute()

,age,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,NHR,HNR,RPDE,DFA,PPE,Shimmer:APQ11,Shimmer:DDA
0,72,5.6431,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.014290,21.640,0.41888,0.54842,0.16006,0.01662,0.04314
1,72,12.6660,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.011112,27.183,0.43493,0.56477,0.10810,0.01689,0.02982
3,72,25.6470,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.027837,24.445,0.48730,0.57794,0.33277,0.01963,0.03317
5,72,40.6520,0.00353,0.000023,0.00119,0.00159,0.00357,0.02227,0.214,0.01006,0.01337,0.009438,22.946,0.53949,0.57243,0.19500,0.02263,0.03019
6,72,47.6490,0.00422,0.000024,0.00212,0.00221,0.00637,0.04352,0.445,0.02376,0.02621,0.013260,22.506,0.49250,0.54779,0.17563,0.03488,0.07128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,61,142.7900,0.00406,0.000031,0.00167,0.00168,0.00500,0.01896,0.160,0.00973,0.01133,0.025137,22.369,0.64215,0.55314,0.21367,0.01549,0.02920
5871,61,149.8400,0.00297,0.000025,0.00119,0.00147,0.00358,0.02315,0.215,0.01052,0.01277,0.011927,22.886,0.52598,0.56518,0.12621,0.01904,0.03157
5872,61,156.8200,0.00349,0.000025,0.00152,0.00187,0.00456,0.02499,0.244,0.01371,0.01456,0.017701,25.065,0.47792,0.57888,0.14157,0.01877,0.04112
5873,61,163.7300,0.00281,0.000020,0.00128,0.00151,0.00383,0.01484,0.131,0.00693,0.00870,0.007984,24.422,0.56865,0.56327,0.14204,0.01307,0.02078


In [14]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 500, random_state = 666, min_samples_leaf=2, max_depth=16)
regressor.fit(X_train.compute(), y_train.compute())


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=666, verbose=0,
                      warm_start=False)

In [15]:
print(regressor.score(X_train, y_train))
print(regressor.score(X_test, y_test))

0.9881774342401901
0.9553061214902394


In [17]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators = 500, random_state = 666, min_samples_leaf=2, max_depth=16)

with joblib.parallel_backend('dask'):
  scores = cross_validate(rfr, X_train.compute(), y_train.compute(), cv=4, return_train_score=True)

scores

{'fit_time': array([20.06600857, 18.10948896, 20.00439072, 20.92196369]),
 'score_time': array([0.16056991, 0.1555841 , 0.13962579, 0.23237801]),
 'test_score': array([-1.52534983, -2.20959639,  0.05944279,  0.396696  ]),
 'train_score': array([0.98980348, 0.98971354, 0.98612305, 0.80187108])}

In [18]:
rf_params = {'n_estimators': [200, 300], 'min_samples_leaf':[ 1, 2], 'max_depth':[ 8, 16, 32]}

rf_model = RandomForestRegressor()

grid_search_rf = GridSearchCV(rf_model,
                              param_grid=rf_params,
                              return_train_score=True)

In [19]:
with joblib.parallel_backend('dask'):
  scores = grid_search_rf.fit(X_train.compute(), y_train.compute())


In [20]:
print("The best params are: ", grid_search_rf.best_params_)

The best params are:  {'max_depth': 16, 'min_samples_leaf': 2, 'n_estimators': 300}


In [21]:
print("The test score is: ", grid_search_rf.score(X_test.compute(), y_test.compute()) )

The test score is:  0.9555975886233157


In [22]:
X

,age,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,NHR,HNR,RPDE,DFA,PPE,Shimmer:APQ11,Shimmer:DDA
npartitions=1,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
Y

Dask Series Structure:
npartitions=1
    float64
        ...
Name: total_UPDRS, dtype: float64
Dask Name: getitem, 4 tasks

In [27]:
import dask.array as da
from dask_ml.decomposition import PCA
import numpy as np

In [28]:
pca = PCA(n_components=8)
dX = da.from_array(np.array(X))
transformed = pca.fit_transform(dX)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X.to_dask_array(lengths=True),
                                                    Y.to_dask_array(lengths=True), test_size=0.2 )

In [30]:
from dask_ml.linear_model import LogisticRegression

lr = LogisticRegression() 
lr.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1.0, max_iter=100, multi_class='ovr',
                   n_jobs=1, penalty='l2', random_state=None, solver='admm',
                   solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

In [31]:
preds_train = lr.predict (X_train.compute()) 
preds_test = lr.predict (X_test.compute())

try:
    print("Training score is: ", roc_auc_score(preds_train, y_train.compute())) 
    print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

except ValueError:
    pass


print(lr.score)

<bound method LogisticRegression.score of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1.0, max_iter=100, multi_class='ovr',
                   n_jobs=1, penalty='l2', random_state=None, solver='admm',
                   solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)>


In [32]:
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test.compute(), y_test.compute())))

Accuracy of logistic regression classifier on test set: 0.00


In [33]:
from dask_ml.linear_model import LinearRegression
from dask_glm.datasets import make_regression

X, y = make_regression()
lr = LinearRegression()
lr.fit(X, y)
lr.predict(X)
lr.predict(X)
lr.score(X, y)


dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [34]:
# Y is the target variable

Y = park["total_UPDRS"]


# X is the feature set

X = park[['age', 'test_time', 'Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP', 'Jitter:PPQ5', 
          'Jitter:DDP', 'Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 
          'NHR', 'HNR', 'RPDE', 'DFA', 'PPE', 'Shimmer:APQ11', 'Shimmer:DDA']]

X_train, X_test, y_train, y_test = train_test_split(X.to_dask_array(lengths=True),
                                                    Y.to_dask_array(lengths=True), test_size=0.2 )

In [35]:
lr = LinearRegression()
lr.fit(X_train.compute(), y_train.compute())
lr.predict(X_train.compute())
lr.predict(X_test.compute())
lr.score(X_train, y_train)


dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [38]:
print(lr.score(X_test.compute(), y_test.compute()))


93.3095226384972


In [39]:
print(lr.score(X_train.compute(), y_train.compute()))


96.20047274789347


Linear Regression appears to have decent results here.

In [ ]:
client.close()